In [ ]:
import cv2
import yaml
from pathlib import Path

IMG_DIR = Path("/kaggle/input/street-sign-set/StreetSignSet/test/images") 
LBL_DIR = Path("/kaggle/input/street-sign-set/StreetSignSet/test/labels") 
OUT_DIR = Path("/kaggle/working/groundTruth/test")

OUT_DIR.mkdir(parents=True, exist_ok=True)

COLORS = {'box': (20, 180, 20), 'text': (255, 255, 255)}
FONT_CFG = {'fontFace': cv2.FONT_HERSHEY_PLAIN, 'fontScale': 1.5, 'thickness': 2}

class_names = ['prio_give_way', 'prio_stop', 'prio_priority_road', 'forb_speed_over_5', 'forb_speed_over_10', 'forb_speed_over_20', 'forb_speed_over_30', 'forb_speed_over_40', 'forb_speed_over_50', 'forb_speed_over_60', 'forb_speed_over_70', 'forb_speed_over_80', 'forb_speed_over_90', 'forb_speed_over_100', 'forb_speed_over_110', 'forb_speed_over_120', 'forb_speed_over_130', 'forb_no_entry', 'forb_no_parking', 'forb_no_stopping', 'forb_overtake_car', 'forb_overtake_trucks', 'forb_trucks', 'forb_turn_left', 'forb_turn_right', 'forb_weight_over_3.5t', 'forb_weight_over_7.5t', 'forb_u_turn', 'info_bus_station', 'info_crosswalk', 'info_highway', 'info_one_way', 'info_parking', 'info_taxi_parking', 'warn_children', 'warn_construction', 'warn_crosswalk', 'warn_cyclists', 'warn_left_curve', 'warn_right_curve', 'warn_domestic_animals', 'warn_other_dangers', 'warn_poor_road_surface', 'warn_roundabout', 'warn_sharp_left_curve', 'warn_sharp_right_curve', 'warn_slippery_road', 'warn_hump', 'warn_traffic_light', 'warn_tram', 'warn_two_way_traffic', 'warn_wild_animals', 'mand_bike_lane', 'mand_go_left', 'mand_go_left_right', 'mand_go_right', 'mand_go_straight', 'mand_go_straight_left', 'mand_go_straight_right', 'mand_pass_left', 'mand_pass_left_right', 'mand_pass_right', 'mand_roundabout']

image_files = [f for f in IMG_DIR.iterdir() if f.suffix.lower() in ('.jpg', '.png', '.jpeg')]

print(f"Trovate {len(image_files)} immagini. Inizio elaborazione...")

for i, img_path in enumerate(image_files, 1):
    lbl_path = LBL_DIR / (img_path.stem + ".txt")
    
    img = cv2.imread(str(img_path))
    if img is None: continue
    h_img, w_img = img.shape[:2]

    if not lbl_path.exists():
        cv2.imwrite(str(OUT_DIR / img_path.name), img)
        continue

    with open(lbl_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        try:
            parts = line.strip().split()
            cls_id = int(parts[0])
            x_c, y_c, w, h = map(float, parts[1:5])
        except (ValueError, IndexError): continue

        w_px, h_px = int(w * w_img), int(h * h_img)
        x_px, y_px = int(x_c * w_img), int(y_c * h_img)
        
        x1 = int(x_px - w_px / 2)
        y1 = int(y_px - h_px / 2)
        x2 = x1 + w_px
        y2 = y1 + h_px

        cv2.rectangle(img, (x1, y1), (x2, y2), COLORS['box'], 2)

        label = class_names[cls_id].upper() if cls_id < len(class_names) else f"UNK_{cls_id}"
        (t_w, t_h), _ = cv2.getTextSize(label, **FONT_CFG)
        
        pad = 5
        bg_y1 = y1 - t_h - (pad * 2) 
        if bg_y1 < 0: bg_y1 = y2
        bg_y1 = min(bg_y1, h_img - t_h - (pad * 2))

        bg_y2 = bg_y1 + t_h + (pad * 2)
        bg_x2 = min(x1 + t_w + (pad * 2), w_img)

        cv2.rectangle(img, (x1, bg_y1), (bg_x2, bg_y2), COLORS['box'], -1)
        cv2.putText(img, label, (x1 + pad, bg_y1 + t_h + pad), 
                    color=COLORS['text'], lineType=cv2.LINE_AA, **FONT_CFG)

    cv2.imwrite(str(OUT_DIR / img_path.name), img)
    print(f"[{i}/{len(image_files)}] Salvato: {img_path.name}")

print(f"\nFinito! Output in: {OUT_DIR}")